In [2]:
import numpy as np
import matplotlib.pyplot as plt
from surrogates import *
import pandas as pd

In [3]:
# read in data

df = pd.read_csv('1ES_binned_lc.csv')
pred_df = pd.read_csv('1ES_pred_DHO_lc.csv')

lc, pred_lc = df['scaled lumin'].values, pred_df['lumin'].values

In [ ]:


# Generate a time series (replace with your actual data)
np.random.seed(0)
# ts = np.random.normal(size=100)

# Create surrogates
white_surrogates = white_noise_surrogates(ts)
correlated_surrogates = correlated_noise_surrogate(ts)
IAAFT_surrogates = IAAFT_surrogates(ts)


# Call the plotting function
surrogate_data = {'white': white_surrogates, 'correlated': correlated_surrogates, 'IAAFT': IAAFT_surrogates}
plot_time_series_and_spectrum(ts, surrogate_data)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

# Generate a time series (replace with your actual data)
np.random.seed(0)
ts = np.random.normal(size=1000)

# Create surrogates
white_surrogates = white_noise_surrogates(ts)
correlated_surrogates = correlated_noise_surrogate(ts)
# IAAFT_surrogates = IAAFT_surrogates(ts)

# Call the plotting function
surrogate_data = {
    'White Noise Surrogates': white_surrogates,
    'Correlated Noise Surrogates': correlated_surrogates,
    # 'IAAFT Surrogates': IAAFT_surrogates 
}
plot_amplitude_and_spectrum(ts, surrogate_data)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate


# Example time series 
ts = np.random.normal(size=100)

# Compute autocorrelation
auto_corr_original = autocorrelation(ts)

# Generate Fourier transform surrogate
surrogate = correlated_noise_surrogate(ts)

# Compute autocorrelation of the surrogate
auto_corr_fourier = autocorrelation(surrogate)

auto_corr_white = autocorrelation(white_noise_surrogates(ts))

# Plot the autocorrelation functions
plt.figure(figsize=(10, 6))
plt.plot(auto_corr_original, label='Original', alpha=0.7)
plt.plot(auto_corr_fourier, label='Fourier Surrogate', alpha=0.7)
plt.plot(auto_corr_white, label='White Noise Surrogate', alpha=0.7)
plt.title('Autocorrelation Function')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.legend()
plt.grid(True)
plt.show()


In [4]:

# Example usage:
ts = np.random.randn(100) 
N = 10  # Number of surrogates

# Generate surrogates
white_noise = white_noise_surrogates(ts, N)
correlated_noise = correlated_noise_surrogate(ts, N)
IAAFT = IAAFT_surrogates(ts, N)

# The shape of each surrogate array will be (N, len(ts))
print("White Noise Surrogates Shape:", white_noise.shape)
print("Correlated Noise Surrogates Shape:", correlated_noise.shape)
print("IAAFT Surrogates Shape:", IAAFT.shape)


White Noise Surrogates Shape: (10, 100)
Correlated Noise Surrogates Shape: (10, 100)
IAAFT Surrogates Shape: (10, 100)


In [5]:
from scipy.stats import mannwhitneyu, ks_2samp
import numpy as np
import matplotlib.pyplot as plt


# Example usage:
ts = np.random.randn(1000)  
surrogate = np.random.randn(1000)

# Calculate the effect size
d = cohens_d(ts, surrogate)

# Calculate the Mann-Whitney U test and KS test
stat, p_value_mw = mannwhitneyu(ts, surrogate)
stat, p_value_ks = ks_2samp(ts, surrogate)

print(f"Cohen's d: {d}")
print(f"Mann-Whitney U Test p-value: {p_value_mw}")
print(f"KS Test p-value: {p_value_ks}")


Cohen's d: -0.01334958586569371
Mann-Whitney U Test p-value: 0.5383041808591436
KS Test p-value: 0.4659595288557257


In [ ]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Define the worker function for parallel execution
def worker(ts):
    G = get_visibility_graph(ts)  # Convert time series to visibility graph
    return compute_basic_graph_measures(G)

# Define a function to execute the computation in parallel
def compute_measures_parallel(ts_array, num_workers=4):
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(worker, ts_array))
    return pd.DataFrame(results)

# Example usage:
# Assuming white_noise_surrogates is an array of shape (100, T)
surrogates = phase_randomized_surrogates(ts, 100)

# Compute graph measures for all surrogates in parallel and store in DataFrame
measures_df = compute_measures_parallel(surrogates)
print(measures_df)



In [ ]:
# compare surrogate measures to original data's measures

def get_pvalues(measures_df, original_measures, larger_is_better=None):
    """
    Calculate p-values for each measure of the original time series.

    :param measures_df: DataFrame where each row is a surrogate and each column is a measure.
    :param original_measures: Dictionary of measures for the original time series.
    :param larger_is_better: Boolean indicating whether larger values of the measure indicate stronger structure.
    :return: Dictionary of p-values for each measure.
    """
    p_values = {}
    num_surrogates = len(measures_df)
    
    if not larger_is_better: # no custom mapping supplied
        larger_is_better = {
            'number_of_nodes': False,  # Number of nodes is usually not directly related to better network structure/topology
            'number_of_edges': False,  # Similar to number_of_nodes, not directly indicative of better structure
            'average_degree': True,  # Higher average degree might indicate a more interconnected or robust network
            'density': True,  # Higher density can indicate a more interconnected network
            'average_clustering_coefficient': True,  # Higher clustering can indicate a more tightly knit, community-like structure
            'average_shortest_path_length': False,  # Shorter path lengths can indicate a more efficient or 'small-world' network
            'diameter': False,  # Similar to average_shortest_path_length, a smaller diameter can indicate efficiency
            'assortativity_coefficient': True,  # Higher assortativity can indicate that nodes preferentially attach to similar nodes, 
                                                # might be a sign of robustness in some networks
        }
        
    for measure in original_measures:
        sorted_values = sorted(measures_df[measure])
        original_value = original_measures[measure]
        
        # Find the number of surrogates that have a smaller (or larger) value than the original
        if larger_is_better[measure]:
            count = sum(value > original_value for value in sorted_values)
        else:
            count = sum(value < original_value for value in sorted_values)
        
        # Calculate the empirical p-value
        p_value = count / num_surrogates
        p_values[measure] = p_value
    
    return p_values

def get_significance(p_values):
    """
    Interpret the p-values and print out the significance level for each measure.

    :param p_values: Dictionary of p-values for each measure.
    """
    for measure, p in p_values.items():
        if p < 0.05:
            print(f"{measure}: significant (p = {p:.3f})")
        elif 0.05 <= p < 0.1:
            print(f"{measure}: moderately significant (p = {p:.3f})")
        else:
            print(f"{measure}: not significant (p = {p:.3f})")

# Example usage:
# Assuming 'original_measures' is a dictionary containing the measures for the original time series
# And 'measures_df' is the DataFrame returned from the compute_measures_parallel function
original_measures = compute_basic_graph_measures(G) 
p_values = get_pvalues(measures_df, original_measures)
get_significance(p_values)



In [ ]:
# Compute mean and standard deviation over the range of recurrence rates for each surrogate
mean_rqa = rqa_df.groupby('surrogate_id').mean()
std_rqa = rqa_df.groupby('surrogate_id').std()

# Compute the 25th and 75th percentiles for each surrogate
q1_rqa = rqa_df.groupby('surrogate_id').quantile(0.25)
q3_rqa = rqa_df.groupby('surrogate_id').quantile(0.75)

# Compute the interquartile range (IQR) for each RQA metric and each surrogate
iqr_rqa = q3_rqa - q1_rqa

print(mean_rqa)
print(std_rqa)
